In [6]:
# !pip3 install numpy pandas matplotlib seaborn scipy scikit-learn statsmodels

In [1]:
import numpy as np
import pandas as pd
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Statistical analysis
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")


In [163]:
productos = pd.read_csv("files/maestro_productos.csv")
clientes = pd.read_excel("files/maestro_clientes.xlsx")
proveedor1 = pd.read_csv("files/sellout-proveedor1.csv", sep = "|")
proveedor2 = pd.read_csv("files/sellout-proveedor2.csv", sep = "|")

## Limpieza

In [166]:
print(clientes.info(), productos.info(), proveedor1.info(), proveedor2.info())
# print(proveedor2.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2452 entries, 0 to 2451
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     2452 non-null   int64 
 1   NUM TDA        2452 non-null   int64 
 2   NOMBRE DE TDA  2452 non-null   object
 3   CADENA         2452 non-null   object
 4   FORMATO        2452 non-null   object
 5   ESTADO         2452 non-null   object
 6   DIVISION       2452 non-null   object
dtypes: int64(2), object(5)
memory usage: 134.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1242 entries, 0 to 1241
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       1242 non-null   int64 
 1   Sku              1242 non-null   int64 
 2   Sku_Description  1242 non-null   object
 3   Sku_Category     1242 non-null   object
 4   Sku_Subcategory  1242 non-null   object
dtypes: int64(2), object(3)
memory

In [167]:
print(clientes.head())

   Unnamed: 0  NUM TDA              NOMBRE DE TDA       CADENA FORMATO  \
0           0        2  Bloody Sea Green BookShop  proveedor_1    Shop   
1           1        4  Funny Sky Blue GipsyTrade  proveedor_1    Shop   
2           2        5      Rare Green Laundrette  proveedor_1    Shop   
3           3        6        Angry Indigo Center  proveedor_1    Shop   
4           4        7           Tall Violet Deli  proveedor_1    Shop   

    ESTADO DIVISION  
0  Coalmar  SURESTE  
1  Coalmar  SURESTE  
2  Coalmar  SURESTE  
3  Coalmar  SURESTE  
4  Coalmar  SURESTE  


In [168]:
print(
clientes['CADENA'].unique(),"\n",
clientes['DIVISION'].unique(),"\n",
clientes['FORMATO'].unique()
)

['proveedor_1' 'proveedor_2'] 
 ['SURESTE' 'SUROESTE' 'PENÍNSULA' 'CENTRO' 'METRO-NORTE' 'BAJÍO'
 'OCCIDENTE' 'CENTRO-SUR' 'METRO-SUR' 'NOROESTE' 'NORESTE'] 
 ['Shop' 'Mall' 'GreatShop' 'Grocery' 'AllInOne' 'Leftorium' 'GipsyTrade'
 'SmallShop' 'TinyShop' 'BigCom' 'HighShop' 'SuperMarket' 'Center']


#### Observaciones clientes

Me aseguré de que en las columnas que parecían clasificatorias no hubiera algún error que clasificara mal a las observaciones, 
por ejemplo, alguna letra de más, una minúscula en vez de mayúscula etc.
Parece que la primer columna es un índice tal vez generado para ellos en su archivo excel

In [169]:
set(clientes.iloc[:, 0].dropna().astype(int)) == set(clientes['NUM TDA'].dropna().astype(int))

False

In [170]:
repetidos = clientes['NUM TDA'].value_counts()
repetidos = repetidos[repetidos > 1]
clientes[clientes['NUM TDA'].isin(repetidos.index.to_list())].sort_values(by='NUM TDA')


,Unnamed: 0,NUM TDA,NOMBRE DE TDA,CADENA,FORMATO,ESTADO,DIVISION
50,50,58,Aquamarine Florist,proveedor_1,Shop,Mamoval,PENÍNSULA
2415,2415,58,Aquamarine Florist,proveedor_2,AllInOne,Marcotho,BAJÍO
115,115,143,Empty Turquoise Baker,proveedor_1,Shop,Egusgul,NOROESTE
235,235,143,Empty Turquoise Baker,proveedor_1,Shop,Nabargus,OCCIDENTE
151,151,240,Dull Navy BookShop,proveedor_1,Shop,Alloralf,METRO-SUR
2430,2430,240,Dull Navy BookShop,proveedor_2,TinyShop,Alloralf,METRO-NORTE
2409,2409,2568,Impressive Coral BigCom,proveedor_2,Grocery,Mobartho,PENÍNSULA
2422,2422,2568,Impressive Coral BigCom,proveedor_2,Grocery,Mobartho,PENÍNSULA


In [171]:
clientes['CADENA'] = clientes['CADENA'].astype(str).str[-1]
clientes.drop(columns=['Unnamed: 0'], inplace=True)
clientes.head()

,NUM TDA,NOMBRE DE TDA,CADENA,FORMATO,ESTADO,DIVISION
0,2,Bloody Sea Green BookShop,1,Shop,Coalmar,SURESTE
1,4,Funny Sky Blue GipsyTrade,1,Shop,Coalmar,SURESTE
2,5,Rare Green Laundrette,1,Shop,Coalmar,SURESTE
3,6,Angry Indigo Center,1,Shop,Coalmar,SURESTE
4,7,Tall Violet Deli,1,Shop,Coalmar,SURESTE


#### Observaciones productos


In [173]:
print(productos.head())

   Unnamed: 0    Sku      Sku_Description   Sku_Category Sku_Subcategory
0           0  11058       Frosted Mango      Sugar-free     Cereal bars
1           1  29207  Nut Cheerios Mango   Super-flavour      Cereal box
2           2  38379    MultiGrain Mango      Sugar-free     Cereal bars
3           3  42671        Crunch Mango      Sugar-free     Cereal bars
4           4  54644          Corn Mango      Sugar-free     Cereal bars


In [174]:
print(
productos['Sku_Category'].unique(),"\n",
productos['Sku_Subcategory'].unique(),"\n"
)

['Sugar-free' 'Super-flavour'] 
 ['Cereal bars' 'Cereal box' 'LECHES UHT'] 



In [175]:
productos['Sku'].unique(), len(productos)

(array([   11058,    29207,    38379, ..., 12409217, 12419224, 12420091],
       shape=(1242,)),
 1242)

In [176]:
productos.drop(productos.columns[0], axis=1, inplace=True)
productos.head()

,Sku,Sku_Description,Sku_Category,Sku_Subcategory
0,11058,Frosted Mango,Sugar-free,Cereal bars
1,29207,Nut Cheerios Mango,Super-flavour,Cereal box
2,38379,MultiGrain Mango,Sugar-free,Cereal bars
3,42671,Crunch Mango,Sugar-free,Cereal bars
4,54644,Corn Mango,Sugar-free,Cereal bars


#### Observaciones proveedor1

In [177]:
proveedor1['Date'] = pd.to_datetime(proveedor1['Date'].str.split(' ').str[0], format='%Y-%m-%d')
proveedor1.head()

,Sku,Date,Store_Id,Units,Price
0,4220015,2017-01-01,18,6.0,15.6
1,4220015,2017-01-01,24,0.0,0.0
2,4220015,2017-01-01,39,0.0,0.0
3,4220015,2017-01-01,44,0.0,0.0
4,4220015,2017-01-01,46,0.0,0.0


### Observaciones proveedor2

In [178]:
print(proveedor2.head())

       Sku        Date  Store_Id  Pieces   Price
0  4240039  2016/01/01      1000    23.0  $16.00
1  4240039  2016/01/01      1001   116.0  $16.00
2  4240039  2016/01/01      1002   146.0  $16.00
3  4240039  2016/01/01      1003    17.0  $16.80
4  4240039  2016/01/01      1004     5.0  $16.70


In [179]:
proveedor2['Price'] = proveedor2['Price'].str.replace('$', '', regex=False).astype(float)

print(proveedor2['Price'].head())

0    16.0
1    16.0
2    16.0
3    16.8
4    16.7
Name: Price, dtype: float64


In [180]:
proveedor2['Date'].dtype 
proveedor2['Date'] = pd.to_datetime(proveedor2['Date'])
proveedor2.rename(columns={'Pieces': 'Units'}, inplace=True)

proveedor2.head()

,Sku,Date,Store_Id,Units,Price
0,4240039,2016-01-01,1000,23.0,16.0
1,4240039,2016-01-01,1001,116.0,16.0
2,4240039,2016-01-01,1002,146.0,16.0
3,4240039,2016-01-01,1003,17.0,16.8
4,4240039,2016-01-01,1004,5.0,16.7


## Validaciones

In [200]:
proveedor1.head()

,Sku,Date,Store_Id,Units,Price
0,4220015,2017-01-01,18,6.0,15.6
1,4220015,2017-01-01,24,0.0,0.0
2,4220015,2017-01-01,39,0.0,0.0
3,4220015,2017-01-01,44,0.0,0.0
4,4220015,2017-01-01,46,0.0,0.0


In [182]:
skus_proveedor1 = proveedor1['Sku'].unique()
skus_proveedor2 = proveedor2['Sku'].unique()
skus_productos = productos['Sku'].unique()

missing_proveedor1 = set(skus_proveedor1) - set(skus_productos)
missing_proveedor2 = set(skus_proveedor2) - set(skus_productos)

missing_proveedor1, missing_proveedor2

({np.int64(5027)}, set())

In [183]:
tiendas_proveedor1 = proveedor1['Store_Id'].unique()
tiendas_proveedor2 = proveedor2['Store_Id'].unique()
tiendas_clientes = clientes['NUM TDA'].unique()

missing_proveedor1 = set(tiendas_proveedor1) - set(tiendas_clientes)
missing_proveedor2 = set(tiendas_proveedor2) - set(tiendas_clientes)

print(missing_proveedor1, "\n", missing_proveedor2)

{np.int64(675), np.int64(676), np.int64(709), np.int64(711), np.int64(712), np.int64(681), np.int64(301), np.int64(174), np.int64(78), np.int64(94)} 
 set()


In [187]:
SKU_ventas = set(proveedor1['Sku'].unique()).union(set(proveedor2['Sku'].unique()))
SKU_sin_ventas = set(productos['Sku'].unique()) - SKU_ventas

print( len(SKU_sin_ventas), len(SKU_ventas), len(proveedor1['Sku'].unique()), len(proveedor2['Sku'].unique()))

1195 48 43 33


In [193]:
divisiones_con_ventas = set(clientes[clientes['NUM TDA'].isin(tiendas_proveedor1)]['DIVISION'].unique()).union(
    set(clientes[clientes['NUM TDA'].isin(tiendas_proveedor2)]['DIVISION'].unique()))
divisiones_sin_ventas = set(clientes['DIVISION'].unique()) - divisiones_con_ventas

divisiones_sin_ventas

set()

In [241]:
# Obtengo fechas únicas
# productos_unicos = productos['Sku'].unique()

fechas = pd.date_range(start=proveedor1['Date'].min(), end=proveedor1['Date'].max(), freq='D')

# Combinaciones posibles de Sku con ventas y Date
combinaciones_completas = pd.MultiIndex.from_product([SKU_ventas, fechas], names=['Sku', 'Date']).to_frame(index=False)

In [263]:
# Combinamos los datos de ventas de ambos proveedores
ventas_totales = pd.concat([proveedor1[['Sku', 'Date', 'Units']], proveedor2[['Sku', 'Date', 'Units']]])

# Unimos las combinaciones de productos y fechas con los datos de ventas en archivos de proveedores
registros_completos = combinaciones_completas.merge(ventas_totales, on=['Sku', 'Date'], how='left')

In [272]:
fecha_min = proveedor1['Date'].min()
fecha_max = proveedor1['Date'].max()
rango_fechas = pd.date_range(start=fecha_min, end=fecha_max)

dias_con_info = registros_completos[registros_completos['Date'].notna()].groupby('Sku')['Date'].nunique().reset_index(name='Dias_con_info')

total_dias = len(rango_fechas)
dias_con_info['completa'] = dias_con_info['Dias_con_info'] == total_dias
dias_con_info


,Sku,Dias_con_info,completa
0,5027,155,True
1,4220015,155,True
2,4230022,155,True
3,4240039,155,True
4,4250046,155,True
5,4260053,155,True
6,4270121,155,True
7,4290336,155,True
8,4300343,155,True
9,4330800,155,True
